In [ ]:
import requests, sqlite3
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def extract_site(web_link):  
  website = web_link
  result = requests.get(website)
  content = result.text
  soup = BeautifulSoup(content, 'lxml')
  return soup

In [ ]:
def extract(web_link, tag_1, tag_2, class_1, class_2, idx_1=None, fnd=True):
  soup = extract_site(web_link)
  if idx_1 == None:
    box = soup.find_all(tag_1, class_=class_1)
  else:
    box = soup.find_all(tag_1, class_=class_1)[idx_1]
  if fnd:
    items = box.find(tag_2, class_=class_2)
    return items
  return box

In [ ]:
def get_items(items, tag_1, tag_2, link_head, idx_1=None, idx_2=None):
  item_dict = {}
  for item in items.find(tag_1).find_all(tag_2)[idx_1:idx_2]:
    item_dict[item.get_text()] = link_head+item.find('a').get('href')
  return item_dict

In [ ]:
def extract_regions(web_link):
  regions = extract(web_link, 'div', 'div', 'content-card-teaser', 'border-left-black', 1)
  region_dict = get_items(regions, 'div', 'strong', 'https://www.cia.gov', idx_1=1)
  return region_dict

In [ ]:
def extract_countries(regions):
  country_dict = {}
  for region in regions:
    countries_grp = extract(regions[region], 'div', '', 'col-md-4', '', fnd=False)
    for countries in countries_grp:
      country_dict.update(get_items(countries, 'ul', 'li', 'https://www.cia.gov'))
  return country_dict

In [ ]:
def get_area(box):
  try:
    box[0].find_all('div')[3]
  except IndexError:
    return {'area': None, 'area_land' : None, 'area_water': None}
  area = box[0].find_all('div')[3]
  data_raw = area.find('p').get_text().split(' ')
  data_clean = []
  for data in data_raw:
    data = data.replace(',', '')
    if can_convert(data):
      data_clean.append(float(data))
  if len(data_clean) < 3:
    for i in range(3-len(data_clean)):
      data_clean.append(None)
  area_dict = {'area': data_clean[0], 'area_land' : data_clean[1], 'area_water': data_clean[2]}
  return area_dict

In [ ]:
def get_population(box):
  try:
    box[1].find_all('div')
  except IndexError:
    return {'population': None,	'population_growth': None, 'birth_rate': None, 'death_rate': None, 'migration_rate': None}
  all_pop = box[1].find_all('div')
  data = all_pop[0].find('p').get_text().split()[0].replace(',', '')
  if can_convert(data):
    pop_lst = [float(data)]
  else:
    pop_lst = [None]
  pop_details = box[1].find_all('div', class_='')
  slice_pop = pop_details[5:20]
  i = get_index(slice_pop)
  if i == None:
    for details in range(4):
      pop_lst.append(None)
  else:
    for details in pop_details[5+i:i+9]:
      data = details.find('p').get_text().split()[0].replace('%', '')
      if can_convert(data):
        pop_lst.append(float(data))
      else:
        pop_lst.append(None)
  pop_dict = {'population': pop_lst[0],	'population_growth': pop_lst[1], 'birth_rate': pop_lst[2], 'death_rate': pop_lst[3], 'migration_rate': pop_lst[4]}
  return pop_dict

In [ ]:
def get_index(lst):
  for i in range(len(lst)):
    item = lst[i].find('h3', class_='mt30')
    if item is not None and item.get_text() == 'Population growth rate':
      return i

In [ ]:
def can_convert(data):
  try:
    float(data)
  except ValueError:
    return False
  return True

In [ ]:
def extract_data(countries_dict):
  data = []
  for country in countries_dict:
    temp_dict = {}
    temp_dict['name'] = country
    soup = extract_site(countries_dict[country])
    box = soup.find_all('div', class_='free-form-content__content')[1:3]
    temp_dict.update(get_area(box))
    temp_dict.update(get_population(box))
    data.append(temp_dict)
    df = pd.DataFrame(data)
  return df

In [ ]:
if __name__ == "__main__" :
  url = "https://www.cia.gov/the-world-factbook/"
  df = extract_data(extract_countries(extract_regions(url)))

In [ ]:
df.head()

,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,Algeria,2381740.0,2381740.0,0.0,44178884.0,1.34,18.52,4.32,-0.82
1,Angola,1246700.0,1246700.0,0.0,34795287.0,3.36,41.80,8.01,-0.19
2,Benin,112622.0,110622.0,2000.0,13754688.0,3.34,41.15,8.01,0.24
3,Botswana,581730.0,566730.0,15000.0,2384246.0,1.40,20.28,9.05,2.81
4,Burkina Faso,274200.0,273800.0,400.0,21935389.0,2.53,33.57,7.71,-0.61


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               261 non-null    object 
 1   area               253 non-null    float64
 2   area_land          250 non-null    float64
 3   area_water         247 non-null    float64
 4   population         237 non-null    float64
 5   population_growth  235 non-null    float64
 6   birth_rate         228 non-null    float64
 7   death_rate         230 non-null    float64
 8   migration_rate     229 non-null    float64
dtypes: float64(8), object(1)
memory usage: 18.5+ KB


In [ ]:
df.isnull().sum()

name                  0
area                  8
area_land            11
area_water           14
population           24
population_growth    26
birth_rate           33
death_rate           31
migration_rate       32
dtype: int64

In [ ]:
df_new = df

In [ ]:
df_new.loc[df_new['area_land']/df_new['area'] > 100000, 'area'] = df_new.loc[df_new['area_land']/df_new['area'] > 100000]['area']*1000000

In [ ]:
df_new

,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,Algeria,2381740.0,2381740.0,0.0,4.417888e+07,1.34,18.52,4.32,-0.82
1,Angola,1246700.0,1246700.0,0.0,3.479529e+07,3.36,41.80,8.01,-0.19
2,Benin,112622.0,110622.0,2000.0,1.375469e+07,3.34,41.15,8.01,0.24
3,Botswana,581730.0,566730.0,15000.0,2.384246e+06,1.40,20.28,9.05,2.81
4,Burkina Faso,274200.0,273800.0,400.0,2.193539e+07,2.53,33.57,7.71,-0.61
...,...,...,...,...,...,...,...,...,...
256,India,3287263.0,2973193.0,314070.0,1.389637e+09,0.67,16.82,10.30,0.16
257,Maldives,298.0,298.0,0.0,3.901640e+05,-0.14,15.54,4.15,-12.78
258,Nepal,147181.0,143351.0,3830.0,3.066660e+07,0.78,17.53,5.58,-4.21
259,Pakistan,796095.0,770875.0,25220.0,2.429238e+08,1.95,26.48,6.02,-0.96


In [ ]:
df_new.to_csv('Factbook.csv')

In [ ]:
conn = sqlite3.connect('factbook.db')
c = conn.cursor()

In [ ]:
c.execute('CREATE TABLE IF NOT EXISTS factbook (name text, area number,	area_land	number, area_water	number,	population	number,	population_growth	number,	birth_rate	number,	death_rate	number,	migration_rate	number)')
conn.commit()

In [ ]:
df_new.to_sql('factbook', conn, if_exists='replace', index = False)

In [ ]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db 

In [ ]:
%%sql

SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,factbook,factbook,2,"CREATE TABLE ""factbook"" (""name"" TEXT, ""area"" REAL, ""area_land"" REAL, ""area_water"" REAL, ""population"" REAL, ""population_growth"" REAL, ""birth_rate"" REAL, ""death_rate"" REAL, ""migration_rate"" REAL)"


In [ ]:
%%sql
SELECT *
FROM factbook

 * sqlite:///factbook.db
Done.


name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
Algeria,2381740.0,2381740.0,0.0,44178884.0,1.34,18.52,4.32,-0.82
Angola,1246700.0,1246700.0,0.0,34795287.0,3.36,41.8,8.01,-0.19
Benin,112622.0,110622.0,2000.0,13754688.0,3.34,41.15,8.01,0.24
Botswana,581730.0,566730.0,15000.0,2384246.0,1.4,20.28,9.05,2.81
Burkina Faso,274200.0,273800.0,400.0,21935389.0,2.53,33.57,7.71,-0.61
Burundi,27830.0,25680.0,2150.0,12696478.0,3.63,35.17,5.96,7.09
Cabo Verde,4033.0,4033.0,0.0,596707.0,1.21,18.49,5.8,-0.57
Cameroon,475440.0,472710.0,2730.0,29321637.0,2.75,35.53,7.73,-0.31
Central African Republic,622984.0,622984.0,0.0,5454533.0,1.78,32.79,11.76,-3.22
Chad,1284000.0,1259200.0,24800.0,17963211.0,3.09,40.45,9.45,-0.13


In [ ]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM factbook;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
50.0,1410539758.0,-2.39,5.91


In [ ]:
%%sql
SELECT *
  FROM factbook
 WHERE population == (SELECT MIN(population)
                        FROM factbook
                     );

 * sqlite:///factbook.db
Done.


name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
Pitcairn Islands,47.0,47.0,0.0,50.0,0.0,None,None,None


In [ ]:
%%sql
SELECT *
  FROM factbook
 WHERE population == (SELECT MAX(population)
                        FROM factbook
                     );

 * sqlite:///factbook.db
Done.


name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
China,9596960.0,9326410.0,270550.0,1410539758.0,0.19,9.93,7.9,-0.11


In [ ]:
%%sql
SELECT name, area, population
  FROM factbook ORDER BY population/area DESC
  LIMIT 10

 * sqlite:///factbook.db
Done.


name,area,population
Macau,28.0,635293.0
Monaco,2.0,31400.0
Singapore,719.0,5921231.0
Hong Kong,1108.0,7276588.0
Gaza Strip,360.0,1997328.0
Gibraltar,7.0,29573.0
Bahrain,760.0,1540558.0
Malta,316.0,464186.0
Bermuda,54.0,72337.0
Sint Maarten,34.0,45126.0


In [ ]:
%%sql
SELECT name, population
  FROM factbook ORDER BY population DESC
  LIMIT 10

 * sqlite:///factbook.db
Done.


name,population
China,1410539758.0
India,1389637446.0
United States,337341954.0
Indonesia,277329163.0
Pakistan,242923845.0
Nigeria,225082083.0
Brazil,217240060.0
Bangladesh,165650475.0
Russia,142021981.0
Mexico,129150971.0


In [ ]:
%%sql
SELECT name, population
 FROM factbook WHERE population IS NOT NULL ORDER BY population ASC
 LIMIT 10

 * sqlite:///factbook.db
Done.


name,population
Pitcairn Islands,50.0
Cocos (Keeling) Islands,596.0
Holy See (Vatican City),1000.0
Paracel Islands,1440.0
Tokelau,1647.0
Norfolk Island,1748.0
Niue,2000.0
Christmas Island,2205.0
Svalbard,2926.0
Falkland Islands (Islas Malvinas),3198.0


In [ ]:
%%sql
SELECT name, population/area as people_per_sq_km
  FROM factbook WHERE population IS NOT NULL AND area > 0 ORDER BY population/area ASC
  LIMIT 10

 * sqlite:///factbook.db
Done.


name,people_per_sq_km
Greenland,0.026680381111368615
Svalbard,0.047159319848497055
Falkland Islands (Islas Malvinas),0.262712560584901
Pitcairn Islands,1.0638297872340425
Mongolia,2.063697961020794
Namibia,3.3087898463165963
Australia,3.376905578190518
Iceland,3.471873786407767
Guyana,3.6734738497178663
Canada,3.8291293553016774


In [ ]:
%%sql
SELECT name, migration_rate
  FROM factbook WHERE migration_rate IS NOT NULL
  ORDER BY migration_rate ASC
  LIMIT 10

 * sqlite:///factbook.db
Done.


name,migration_rate
American Samoa,-29.8
Cook Islands,-27.31
"Micronesia, Federated States of",-20.91
Tonga,-18.01
Northern Mariana Islands,-13.6
Maldives,-12.78
Puerto Rico,-11.9
Jordan,-11.08
Guam,-10.96
Nauru,-10.6


In [ ]:
%%sql
SELECT name, migration_rate
  FROM factbook WHERE migration_rate IS NOT NULL
  ORDER BY migration_rate DESC
  LIMIT 10

 * sqlite:///factbook.db
Done.


name,migration_rate
Syria,40.58
South Sudan,20.97
Equatorial Guinea,13.96
Venezuela,13.88
British Virgin Islands,13.56
Cayman Islands,12.56
Luxembourg,12.01
Anguilla,10.67
Monaco,10.03
Turks and Caicos Islands,8.58


In [ ]:
%%sql
SELECT name, birth_rate
  FROM factbook WHERE birth_rate IS NOT NULL
  ORDER BY birth_rate DESC
  LIMIT 10

 * sqlite:///factbook.db
Done.


name,birth_rate
Niger,47.08
Angola,41.8
Benin,41.15
Mali,41.07
Uganda,40.94
Chad,40.45
"Congo, Democratic Republic of the",40.08
Somalia,37.98
South Sudan,37.69
Mozambique,37.47


In [ ]:
%%sql
SELECT name, birth_rate
  FROM factbook WHERE birth_rate IS NOT NULL
  ORDER BY birth_rate ASC
  LIMIT 10

 * sqlite:///factbook.db
Done.


name,birth_rate
Saint Pierre and Miquelon,6.47
Monaco,6.66
Andorra,6.88
"Korea, South",6.92
Japan,6.95
Italy,6.95
Spain,7.13
Taiwan,7.39
Greece,7.61
Puerto Rico,7.87


In [ ]:
%%sql
SELECT name, death_rate
  FROM factbook WHERE death_rate IS NOT NULL
  ORDER BY death_rate DESC
  LIMIT 10

 * sqlite:///factbook.db
Done.


name,death_rate
Serbia,16.39
Romania,15.26
Lithuania,15.12
Latvia,14.65
Bulgaria,14.41
Ukraine,13.77
Russia,13.36
Estonia,13.1
Belarus,12.88
Croatia,12.88


In [ ]:
%%sql
SELECT name, death_rate
  FROM factbook WHERE death_rate IS NOT NULL
  ORDER BY death_rate ASC
  LIMIT 10

 * sqlite:///factbook.db
Done.


name,death_rate
Tokelau,-3.84
Qatar,1.42
United Arab Emirates,1.56
Kuwait,2.25
Bahrain,2.82
Gaza Strip,2.91
Oman,3.23
West Bank,3.4
Saudi Arabia,3.42
Libya,3.45
